In [79]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from seaborn import set_style
set_style('whitegrid')

In [80]:
gross_data = pd.read_csv('https://raw.githubusercontent.com/tacookson/data/master/broadway-grosses/grosses.csv',parse_dates = ['week_ending'])

In [81]:
show_data = pd.read_csv('https://raw.githubusercontent.com/PSha98/Erdos-2022-project/main/Data_collection/show_info_1985-2019.csv')

In [82]:
gross_data


,week_ending,week_number,weekly_gross_overall,show,theatre,weekly_gross,potential_gross,avg_ticket_price,top_ticket_price,seats_sold,seats_in_theatre,pct_capacity,performances,previews
0,1985-06-09,1,3915937.00,42nd Street,St. James Theatre,282368.00,NaN,30.42,NaN,9281,1655,0.7010,8,0
1,1985-06-09,1,3915937.00,A Chorus Line,Sam S. Shubert Theatre,222584.00,NaN,27.25,NaN,8167,1472,0.6935,8,0
2,1985-06-09,1,3915937.00,Aren't We All?,Brooks Atkinson Theatre,249272.00,NaN,33.75,NaN,7386,1088,0.8486,8,0
3,1985-06-09,1,3915937.00,Arms and the Man,Circle in the Square Theatre,95688.00,NaN,20.87,NaN,4586,682,0.8405,8,0
4,1985-06-09,1,3915937.00,As Is,Lyceum Theatre,61059.00,NaN,20.78,NaN,2938,684,0.5369,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47519,2020-03-01,40,26109896.25,The Phantom of the Opera,Majestic Theatre,639215.93,1358986.0,72.18,213.0,8856,1605,0.6897,8,0
47520,2020-03-01,40,26109896.25,Tina: The Tina Turner Musical,Lunt-Fontanne Theatre,1320766.00,1566688.0,132.02,297.0,10004,1478,0.8461,8,0
47521,2020-03-01,40,26109896.25,To Kill A Mockingbird,Sam S. Shubert Theatre,1132278.54,1549625.0,115.41,423.0,9811,1435,0.9767,7,0
47522,2020-03-01,40,26109896.25,West Side Story,Broadway Theatre,1598947.32,1722464.0,114.87,373.0,13920,1740,1.0000,8,0


In [83]:
#0.5)  Use consumer price index to normalize dollar values. Used this data source https://fred.stlouisfed.org/series/CPIAUCSL 
CPI_data = pd.read_csv('https://raw.githubusercontent.com/PSha98/Erdos-2022-project/main/Data_collection/CPI_USBLS.csv', parse_dates = ['DATE'])
CPI_data.rename(columns = {'CWSR0000SA0':'CPI'}, inplace = True)
gross_data['year'] = gross_data['week_ending'].dt.year
gross_data['month'] = gross_data['week_ending'].dt.month
CPI_data['year'] = CPI_data['DATE'].dt.year
CPI_data['month']= CPI_data['DATE'].dt.month

In [84]:
## All values in  2022-04-01 dollar value
CPI_data['CPI'] = CPI_data.iloc[-1]['CPI']/CPI_data['CPI']

In [85]:
CPI_data2 = CPI_data[['CPI','month','year']]
gross_data2 = gross_data.merge(CPI_data2, how = 'left', on = ['month', 'year'])
gross_data['weekly_gross_overall'] = gross_data2['weekly_gross_overall']*gross_data2['CPI']
gross_data['weekly_gross'] = gross_data2['weekly_gross']*gross_data2['CPI']
gross_data['avg_ticket_price'] = gross_data2['avg_ticket_price']*gross_data2['CPI']
gross_data['potential_gross'] = gross_data2['potential_gross']*gross_data2['CPI']
gross_data['top_ticket_price']= gross_data2['top_ticket_price']*gross_data2['CPI']

In [8]:
#Reorganize data set for regression. 

In [86]:
import datetime
#df = gross_data[gross_data['year']>=1997]
#df['gross_ratio']
var = 'weekly_gross'
df = gross_data.groupby(['show','theatre']).filter(lambda x : len(x)>4).copy()
show_type = pd.read_csv('show_data_final.csv',parse_dates = ['opening_date'])
show_type[['name','theatre']].duplicated()
df1 = show_type[~show_type[['name','theatre']].duplicated()]
df1 = df1[df1['performances']+df1['previews'] >= 32].copy()
df1[['w1','w2','w3','w4','year1','6months','3months','length']] = ['0','0','0','0','0','0','0','0']
reg_data = df1.copy()
for i, r in df1[['name','theatre']].iterrows():
    data = df[(df['show'] == r['name'])& (df['theatre'] == r['theatre'])]
    isempty = data.empty
    if isempty == False:
        data.sort_values(by = 'week_ending')
        init = data['week_ending'].min()
        d = datetime.timedelta(weeks=4)
        d2 = datetime.timedelta(weeks=48)
        d3 = datetime.timedelta(weeks = 8)
        d4 = datetime.timedelta(weeks = 20)
        date1 = init + d
        date2 = date1 + d2
        date3 = date1 + d3
        date4 = date1 + d4
        reg_data['w1'].loc[(reg_data['name'] == r['name']) & (reg_data['theatre'] == r['theatre'])] = data[var].iloc[0]
        reg_data['w2'].loc[(reg_data['name'] == r['name']) & (reg_data['theatre'] == r['theatre'])] = data[var].iloc[1]
        reg_data['w3'].loc[(reg_data['name'] == r['name']) & (reg_data['theatre'] == r['theatre'])] = data[var].iloc[2]
        reg_data['w4'].loc[(reg_data['name'] == r['name']) & (reg_data['theatre'] == r['theatre'])] = data[var].iloc[3]
        reg_data['year1'].loc[(reg_data['name'] == r['name']) & (reg_data['theatre'] == r['theatre'])] = data[var].loc[(data['week_ending'] >= init) & (data['week_ending'] <=  date2)].sum()  
        reg_data['6months'].loc[(reg_data['name'] == r['name']) & (reg_data['theatre'] == r['theatre'])] = data[var].loc[(data['week_ending'] >= init) & (data['week_ending'] <=  date4)].sum()                        
        reg_data['3months'].loc[(reg_data['name'] == r['name']) & (reg_data['theatre'] == r['theatre'])] = data[var].loc[(data['week_ending'] >= init) & (data['week_ending'] <=  date3)].sum()
        reg_data['length'].loc[(reg_data['name'] == r['name']) & (reg_data['theatre'] == r['theatre'])] = len(data)
        

/Users/gautam/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [87]:
reg_data
reg_data['month'] = reg_data['opening_date'].dt.month

In [88]:
a =  pd.get_dummies(reg_data['month'])
reg_data[['m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11']] = a[[1,2,3,4,5,6,7,8,9,10,11]]
                                                                                                                
                                                                                                                     
                                                                                                                     
                                                                                                                     
                                                                                                                     
                                                                                                                     
                                                                                                                     

In [89]:
for col in reg_data.columns:
    print(col)

Unnamed: 0
season
opening_date
name
theatre
type
all_nominations
all_Wins
tony_best_musical_nom
tony_best_musical_winner
tony_best_revival_musical_nom
tony_best_revival_musical_winner
tony_best_play_nom
tony_best_play_winner
tony_best_revival_play_nom
tony_best_revival_play_winner
performances
previews
play
musical
revival
w1
w2
w3
w4
year1
6months
3months
length
month
m1
m2
m3
m4
m5
m6
m7
m8
m9
m10
m11


In [90]:
for i in ['type',
'all_nominations',
'all_Wins',
'tony_best_musical_nom',
'tony_best_musical_winner',
'tony_best_revival_musical_nom',
'tony_best_revival_musical_winner',
'tony_best_play_nom',
'tony_best_play_winner',
'tony_best_revival_play_nom',
'tony_best_revival_play_winner',
'previews']:
    reg_data.pop(i)

In [101]:
reg_data['length']
df7 = reg_data
reg_data.loc[reg_data['length'] >= 0]

TypeError: '>=' not supported between instances of 'str' and 'int'

In [102]:
from sklearn.model_selection import train_test_split

In [50]:
reg_train, reg_test = train_test_split(reg_data.copy(),
                                        shuffle = True,
                                        random_state = 440,
                                        test_size=.2)

In [51]:
X_train = reg_train[['w1',
'w2',
'w3',
'w4',
'musical',
'revival',
1,
2,
3,
4,
5,
6,
7,
8,
9,
10,
11                     
]].values

In [52]:
X_train

array([[337722.9330867052, 377598.4511445087, 308695.8412023122, ..., 0,
        0, 0],
       [246989.31589285715, 655862.3983883604, 698299.5064801343, ..., 0,
        0, 0],
       [687287.4111496063, 766838.7378476042, 710890.0400314219, ..., 0,
        0, 0],
       ...,
       [1185661.4809643233, 1136121.2677849052, 1601801.4192133688, ...,
        0, 0, 0],
       [520850.5235958347, 1016227.7460763501, 1072297.5276797777, ...,
        0, 0, 0],
       [462658.26888072415, 876578.4617451098, 791232.0209282562, ..., 0,
        0, 0]], dtype=object)

In [53]:
y_train =  reg_train['length'].values
y_train

array([12, 19, 50, '0', 11, 118, 70, 14, 6, 10, 17, 56, 47, 25, 13, '0',
       33, 17, 30, 30, 12, 10, 20, 17, 27, 17, 19, 15, 13, 75, 56, 15, 11,
       47, 12, 26, 6, '0', 23, 13, '0', 20, 12, 37, 167, 10, 65, 11, 30,
       91, 28, 12, 17, 9, 13, 11, 51, 61, 65, 38, '0', 44, 10, '0', 15, 9,
       14, 95, 5, 25, '0', 11, 10, 10, 102, 10, 11, 12, '0', 94, 10, '0',
       11, 16, 19, 14, 16, 358, 14, 10, 11, 17, 7, 65, 42, 7, 48, 11, 13,
       81, 20, 17, 7, 7, 19, 5, 12, 25, 55, 8, 5, 10, 40, '0', 10, 11, 5,
       11, 14, 11, 23, 7, 5, 12, 14, '0', 7, 18, '0', 10, 59, 14, 8, 16,
       13, 26, 37, 17, 13, '0', 13, 11, 9, 15, 12, 14, '0', '0', 19, 25,
       12, 43, 17, '0', 20, 40, 16, 31, 10, 15, 64, 18, 11, 34, 22, 17,
       14, 19, 15, 56, 79, 31, 121, 6, 9, 19, 19, 96, 13, 27, 108, 8, 20,
       6, 11, 9, 6, 17, 171, 25, 25, 13, 75, 15, 87, 61, 20, 15, 20, 318,
       11, 11, '0', '0', 33, 14, 26, 78, 23, 12, '0', 29, 6, 95, 11, 14,
       '0', 17, 7, 51, 9, 21, 17, 15, 11, 9

In [54]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression(copy_X=True, fit_intercept=True)


In [55]:
reg.fit(X_train, y_train)

LinearRegression()

In [56]:
reg.coef_

array([-4.20595512e-05,  9.59749635e-05, -4.28562251e-05,  4.37148272e-05,
        4.76279753e+01, -2.22398737e+01,  4.59277419e+01,  8.01809460e+00,
        1.18531619e+01,  9.82022598e+00,  1.91235272e+01,  1.70495059e+01,
        1.26885768e+01,  1.47218106e+01,  4.23493200e+00,  9.07731332e+00,
        1.24034551e+01])

In [57]:
y_pred_sklearn = reg.predict(X_train)

In [72]:
y_train.astype(np.float32)

array([  12.,   19.,   50.,    0.,   11.,  118.,   70.,   14.,    6.,
         10.,   17.,   56.,   47.,   25.,   13.,    0.,   33.,   17.,
         30.,   30.,   12.,   10.,   20.,   17.,   27.,   17.,   19.,
         15.,   13.,   75.,   56.,   15.,   11.,   47.,   12.,   26.,
          6.,    0.,   23.,   13.,    0.,   20.,   12.,   37.,  167.,
         10.,   65.,   11.,   30.,   91.,   28.,   12.,   17.,    9.,
         13.,   11.,   51.,   61.,   65.,   38.,    0.,   44.,   10.,
          0.,   15.,    9.,   14.,   95.,    5.,   25.,    0.,   11.,
         10.,   10.,  102.,   10.,   11.,   12.,    0.,   94.,   10.,
          0.,   11.,   16.,   19.,   14.,   16.,  358.,   14.,   10.,
         11.,   17.,    7.,   65.,   42.,    7.,   48.,   11.,   13.,
         81.,   20.,   17.,    7.,    7.,   19.,    5.,   12.,   25.,
         55.,    8.,    5.,   10.,   40.,    0.,   10.,   11.,    5.,
         11.,   14.,   11.,   23.,    7.,    5.,   12.,   14.,    0.,
          7.,   18.,

In [76]:
print("the mse is", np.sum(np.power(y_train-y_pred_sklearn, 2))/len(y_train))

TypeError: unsupported operand type(s) for -: 'str' and 'float'